In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pickle
import seaborn as sns
import time
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from numpy import concatenate
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.semi_supervised import LabelPropagation
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score, mean_squared_error

In [3]:
data = pd.read_csv('~/Desktop/716/javaperformance/java-csv.csv')
data.head(5)

,QueryString,PredictedLabel
0,64 bit java,False
1,java 64 bit download,False
2,java test,False
3,java control panel windows 10,False
4,java jre download,False


In [4]:
data["len"] = [len(text) for text in data["QueryString"].values]
data.groupby("PredictedLabel")["len"].mean()

PredictedLabel
False    19.187052
True     21.614267
Name: len, dtype: float64

In [5]:
data.head()

,QueryString,PredictedLabel,len
0,64 bit java,False,11
1,java 64 bit download,False,20
2,java test,False,9
3,java control panel windows 10,False,29
4,java jre download,False,17


In [6]:
def cleanText(text):
    
    lemma = WordNetLemmatizer()
    stp = stopwords.words('english')
    
    text = re.sub("[^a-zA-Z0-9]"," ",text)
    text = text.lower()

    text = nltk.word_tokenize(text)
    text = [lemma.lemmatize(word) for word in text]
    
    text = [word for word in text if word not in stp]
    
    text = " ".join(text)
    
    return text

In [7]:
cleanedText = []
for text in data["QueryString"]:   
    cleanedText.append(cleanText(text))
print(cleanedText[0])

64 bit java


In [8]:
data["PredictedLabel"].value_counts()

False    4325
True     2271
Name: PredictedLabel, dtype: int64

In [9]:
y = data["PredictedLabel"]
y.shape

(6596,)

In [10]:
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(cleanedText).toarray()
x.shape

(6596, 1886)

In [11]:
print(np.histogram(x))

(array([12422102,     4712,      292,      666,     1822,     2375,
           2541,     2107,     1129,     2310], dtype=int64), array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]))


In [12]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=4,test_size=0.3)
X_train_lab, X_test_unlab, y_train_lab, y_test_unlab = train_test_split(x_train, y_train, test_size=0.50, random_state=1, stratify=y_train)

In [13]:
#classifier = XGBClassifier(objective= 'reg:squaredlogerror', use_label_encoder=False)
#classifier = SVC()
classifier = classifier = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100)
classifier.fit(x_train,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100)

In [15]:
y_pred = classifier.predict(x_test)
score = accuracy_score(y_test, y_pred)
print('Accuracy: %.3f' % (score*100))

Accuracy: 84.487


In [68]:

#model = LogisticRegression()
#model = SVC()
#model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100)
#model.fit(X_train_lab, y_train_lab)

In [69]:
#yhat = model.predict(x_test)

#score = accuracy_score(y_test, yhat)
#print('Accuracy: %.3f' % (score*100))

In [16]:
nolabel = [-1 for _ in range(len(y_test_unlab))]

y_train_mixed = concatenate((y_train_lab, nolabel))

model = LabelPropagation()

model.fit(x_train, y_train)

yhat = model.predict(x_test)

score = accuracy_score(y_test, yhat)

print('Accuracy: %.3f' % (score*100))

Accuracy: 88.833


In [17]:
tran_labels = model.transduction_

In [60]:
#model2 = SVC()
model2 = LogisticRegression()
model2.fit(x_train, tran_labels)

y_hat = model2.predict(x_test)
score2 = accuracy_score(y_test, y_hat)
print('Accuracy: %.3f' % (score2*100))

Accuracy: 90.450


In [ ]:
#x.astype(numpy.float32)

In [61]:
y_pred = y_hat.astype(np.float32)
y_test = y_test.astype(np.float32)

test_mse = mean_squared_error(y_test, y_pred)

print('MSE test data: {:.5}'.format(test_mse))
print('RMSE test data: {:.5}'.format(
    np.sqrt(np.sqrt(np.absolute(test_mse)))))
print('R2 test data: {:.5}'.format(
    r2_score(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
print(cm)
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]
print(TP)
print(FP)
print(FN)
print(TN)
accuracy = (TP+TN)/(TP+FP+FN+TN)
print('Accuracy = ',accuracy)

Precision = (TP)/(TP+FP)
print('Precision = ',Precision)

Recall = (TP)/(TP+FN)
print('Recall = ',Recall)

f1score = 2*(Recall*Precision)/(Recall+Precision)
print('f1score = ',f1score)


MSE test data: 0.095503
RMSE test data: 0.55591
R2 test data: 0.57626
[[1280   20]
 [ 169  510]]
1280
20
169
510
Accuracy =  0.9044972208185953
Precision =  0.9846153846153847
Recall =  0.883367839889579
f1score =  0.9312477264459804


# Other single classifier models

In [62]:
model3 = SGDClassifier()
model3.fit(X_train_lab, y_train_lab)

y_hat3 = model3.predict(x_test)
score3 = accuracy_score(y_test, y_hat3)
print('Accuracy: %.3f' % (score3*100))

Accuracy: 90.298


In [63]:
y_pred = y_hat3.astype(np.float32)
y_test = y_test.astype(np.float32)

test_mse = mean_squared_error(y_test, y_pred)

print('MSE test data: {:.5}'.format(test_mse))
print('RMSE test data: {:.5}'.format(
    np.sqrt(np.sqrt(np.absolute(test_mse)))))
print('R2 test data: {:.5}'.format(
    r2_score(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
print(cm)
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]
print(TP)
print(FP)
print(FN)
print(TN)
accuracy = (TP+TN)/(TP+FP+FN+TN)
print('Accuracy = ',accuracy)

Precision = (TP)/(TP+FP)
print('Precision = ',Precision)

Recall = (TP)/(TP+FN)
print('Recall = ',Recall)

f1score = 2*(Recall*Precision)/(Recall+Precision)
print('f1score = ',f1score)


MSE test data: 0.097019
RMSE test data: 0.5581
R2 test data: 0.56954
[[1240   60]
 [ 132  547]]
1240
60
132
547
Accuracy =  0.9029813036887316
Precision =  0.9538461538461539
Recall =  0.9037900874635568
f1score =  0.9281437125748503


In [50]:
model4 = XGBClassifier(use_label_encoder=False, objective= 'reg:squaredlogerror')
model4.fit(X_train_lab, y_train_lab)

y_hat4 = model4.predict(x_test)
score4 = accuracy_score(y_test, y_hat4)
print('Accuracy: %.3f' % (score4*100))

Accuracy: 88.732


In [64]:
y_pred = y_hat4.astype(np.float32)
y_test = y_test.astype(np.float32)

test_mse = mean_squared_error(y_test, y_pred)

print('MSE test data: {:.5}'.format(test_mse))
print('RMSE test data: {:.5}'.format(
    np.sqrt(np.sqrt(np.absolute(test_mse)))))
print('R2 test data: {:.5}'.format(
    r2_score(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
print(cm)
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]
print(TP)
print(FP)
print(FN)
print(TN)
accuracy = (TP+TN)/(TP+FP+FN+TN)
print('Accuracy = ',accuracy)

Precision = (TP)/(TP+FP)
print('Precision = ',Precision)

Recall = (TP)/(TP+FN)
print('Recall = ',Recall)

f1score = 2*(Recall*Precision)/(Recall+Precision)
print('f1score = ',f1score)

MSE test data: 0.11268
RMSE test data: 0.57938
R2 test data: 0.50004
[[1280   20]
 [ 203  476]]
1280
20
203
476
Accuracy =  0.8873168266801414
Precision =  0.9846153846153847
Recall =  0.8631153068105192
f1score =  0.9198706431908013


In [65]:
model5 = LogisticRegression()
model5.fit(X_train_lab, y_train_lab)

y_hat5 = model5.predict(x_test)
score5 = accuracy_score(y_test, y_hat5)
print('Accuracy: %.3f' % (score5*100))

Accuracy: 88.580


In [66]:
y_pred = y_hat5.astype(np.float32)
y_test = y_test.astype(np.float32)

test_mse = mean_squared_error(y_test, y_pred)

print('MSE test data: {:.5}'.format(test_mse))
print('RMSE test data: {:.5}'.format(
    np.sqrt(np.sqrt(np.absolute(test_mse)))))
print('R2 test data: {:.5}'.format(
    r2_score(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
print(cm)
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]
print(TP)
print(FP)
print(FN)
print(TN)
accuracy = (TP+TN)/(TP+FP+FN+TN)
print('Accuracy = ',accuracy)

Precision = (TP)/(TP+FP)
print('Precision = ',Precision)

Recall = (TP)/(TP+FN)
print('Recall = ',Recall)

f1score = 2*(Recall*Precision)/(Recall+Precision)
print('f1score = ',f1score)

MSE test data: 0.1142
RMSE test data: 0.58132
R2 test data: 0.49331
[[1285   15]
 [ 211  468]]
1285
15
211
468
Accuracy =  0.8858009095502779
Precision =  0.9884615384615385
Recall =  0.858957219251337
f1score =  0.9191702432045781


In [67]:
model7 = MultinomialNB()
model7.fit(X_train_lab, y_train_lab)

y_hat7 = model7.predict(x_test)
score7 = accuracy_score(y_test, y_hat7)
print('Accuracy: %.3f' % (score7*100))

Accuracy: 88.125


In [68]:
y_pred = y_hat7.astype(np.float32)
y_test = y_test.astype(np.float32)

test_mse = mean_squared_error(y_test, y_pred)

print('MSE test data: {:.5}'.format(test_mse))
print('RMSE test data: {:.5}'.format(
    np.sqrt(np.sqrt(np.absolute(test_mse)))))
print('R2 test data: {:.5}'.format(
    r2_score(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
print(cm)
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]
print(TP)
print(FP)
print(FN)
print(TN)
accuracy = (TP+TN)/(TP+FP+FN+TN)
print('Accuracy = ',accuracy)

Precision = (TP)/(TP+FP)
print('Precision = ',Precision)

Recall = (TP)/(TP+FN)
print('Recall = ',Recall)

f1score = 2*(Recall*Precision)/(Recall+Precision)
print('f1score = ',f1score)

MSE test data: 0.11875
RMSE test data: 0.58702
R2 test data: 0.47313
[[1221   79]
 [ 156  523]]
1221
79
156
523
Accuracy =  0.8812531581606872
Precision =  0.9392307692307692
Recall =  0.8867102396514162
f1score =  0.9122151662308553


In [69]:
model8 = DecisionTreeClassifier()
model8.fit(X_train_lab, y_train_lab)

y_hat8 = model8.predict(x_test)
score8 = accuracy_score(y_test, y_hat8)
print('Accuracy: %.3f' % (score8*100))

Accuracy: 89.490


In [70]:
y_pred = y_hat8.astype(np.float32)
y_test = y_test.astype(np.float32)

test_mse = mean_squared_error(y_test, y_pred)

print('MSE test data: {:.5}'.format(test_mse))
print('RMSE test data: {:.5}'.format(
    np.sqrt(np.sqrt(np.absolute(test_mse)))))
print('R2 test data: {:.5}'.format(
    r2_score(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
print(cm)
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]
print(TP)
print(FP)
print(FN)
print(TN)
accuracy = (TP+TN)/(TP+FP+FN+TN)
print('Accuracy = ',accuracy)

Precision = (TP)/(TP+FP)
print('Precision = ',Precision)

Recall = (TP)/(TP+FN)
print('Recall = ',Recall)

f1score = 2*(Recall*Precision)/(Recall+Precision)
print('f1score = ',f1score)

MSE test data: 0.1051
RMSE test data: 0.56938
R2 test data: 0.53367
[[1231   69]
 [ 139  540]]
1231
69
139
540
Accuracy =  0.8948964123294593
Precision =  0.946923076923077
Recall =  0.8985401459854014
f1score =  0.9220973782771537
